<a href="https://colab.research.google.com/github/aetev/Learning-stuff-/blob/main/yf%20test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.lib.stride_tricks import sliding_window_view
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [2]:
# Calculate the date 730 days ago
end_date = datetime.now()
start_date = end_date - timedelta(days=729)

# Download the data
data = yf.download('USDJPY=X', start=start_date, end=end_date, interval='1h')

# Print the data
print(data)

[*********************100%%**********************]  1 of 1 completed
                                 Open        High         Low       Close  \
Datetime                                                                    
2021-10-18 07:00:00+01:00  114.316002  114.349998  114.209999  114.320000   
2021-10-18 08:00:00+01:00  114.325996  114.457001  114.220001  114.295998   
2021-10-18 09:00:00+01:00  114.293999  114.400002  114.199997  114.365997   
2021-10-18 10:00:00+01:00  114.360001  114.413002  114.279999  114.379997   
2021-10-18 11:00:00+01:00  114.383003  114.440002  114.290001  114.330002   
...                               ...         ...         ...         ...   
2023-10-16 16:00:00+01:00  149.639008  149.645004  149.546005  149.572998   
2023-10-16 17:00:00+01:00  149.570007  149.610001  149.544006  149.578995   
2023-10-16 18:00:00+01:00  149.578995  149.613007  149.542007  149.589005   
2023-10-16 19:00:00+01:00  149.587997  149.598007  149.511993  149.514008   
2023-10

In [3]:
High = data['High'].to_numpy()
Close = data['Close'].to_numpy()
Low = data['Low'].to_numpy()
Std = np.std(np.diff(Close))
Close_Diff = np.diff(Close)/Std
High_Adj = (High[1:]-Close[1:])/Std
Low_Adj = (Close[1:]-Low[1:])/Std

In [13]:
stack = np.stack((Close_Diff, High_Adj, Low_Adj), axis=1)
print(stack)

[[-0.13078083  0.87726249  0.41408386]
 [ 0.38140943  0.18527977  0.90449117]
 [ 0.07628189  0.17983403  0.54486468]
 ...
 [ 0.05454051  0.13078083  0.25607435]
 [-0.40863812  0.45769132  0.01097461]
 [ 0.17975089  0.20161698  0.1907255 ]]


In [5]:

# Define the window size and shift
window_size = 10
shift = 1

# Create the sliding window dataset
dataset = tf.keras.utils.timeseries_dataset_from_array(
    stack,
    targets=None,  # No targets provided, as we are only interested in the input
    sequence_length=window_size,
    sequence_stride=shift,
    batch_size=None,
    shuffle=False  # Set to True if you want to shuffle the windows
)

print(dataset.take(0))

<_TakeDataset element_spec=TensorSpec(shape=(None, 3), dtype=tf.float64, name=None)>


In [8]:
window_view = np.lib.stride_tricks.sliding_window_view(stack, (window_size,),axis=0)

In [29]:
print(window_view[0])
input = window_view[:-1]
target = Close_Diff[10:]
print(target)

[[-0.13078083  0.38140943  0.07628189 -0.27241156  0.22884566 -0.51219026
  -0.35962649  0.14711804 -0.08717336  0.        ]
 [ 0.87726249  0.18527977  0.17983403  0.59936362  0.17434672  0.55575616
   0.60480936  0.29423607  0.43046264  0.27245313]
 [ 0.41408386  0.90449117  0.54486468  0.21795419  0.6102551   1.02438052
   0.39230091  0.70287419  0.17979246  0.17979246]]
[-0.07083615  0.27245313  0.3269105  ...  0.05454051 -0.40863812
  0.17975089]


In [ ]:
plt.plot(Low_Adj)

In [ ]:
def Adj_Data(DataFrame):
  High = data['High'].to_numpy()
  Close = data['Close'].to_numpy()
  Low = data['Low'].to_numpy()
  Std = np.std(np.diff(Close))
  Close_Diff = np.diff(Close)/Std
  High_Adj = (High[1:]-Close[1:])/Std
  Low_Adj = (Close[1:]-Low[1:])/Std